In [2]:
import numpy as np
import pandas as pd
import math
import statsmodels.api as sm

pd.options.mode.copy_on_write = True
import random
from matplotlib.pyplot import subplots
import matplotlib.pyplot as plt
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize)

In [3]:
from ISLP import confusion_table
from ISLP.models import contrast
from sklearn.discriminant_analysis import \
     (LinearDiscriminantAnalysis as LDA,
      QuadraticDiscriminantAnalysis as QDA)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import RocCurveDisplay
roc_curve_est = RocCurveDisplay.from_estimator 
roc_curve_pred = RocCurveDisplay.from_predictions 
import statsmodels.api as sm


from sklearn.tree import (DecisionTreeClassifier as DTC,
                          DecisionTreeRegressor as DTR,
                          plot_tree,
                          export_text)
from sklearn.metrics import (accuracy_score,
                             log_loss)
from sklearn.ensemble import \
     (RandomForestRegressor as RF,
      RandomForestClassifier as RFC,
      GradientBoostingRegressor as GBR,
      GradientBoostingClassifier as GBC
)

from functools import partial
from sklearn.model_selection import \
     (cross_validate,
      KFold,
      ShuffleSplit)
from sklearn.model_selection import cross_val_score
from sklearn.base import clone
from ISLP.models import sklearn_sm

from sklearn.cluster import \
     (KMeans,
      AgglomerativeClustering)
from scipy.cluster.hierarchy import \
     (dendrogram,
      cut_tree)
from ISLP.cluster import compute_linkage

from sklearn.metrics import confusion_matrix, accuracy_score

In [4]:
new_mega_UCL = pd.read_csv('../Data/key_stats.csv')
new_mega_UCL

,player_name,club,position,minutes_played,match_played,goals,assists,distance_covered
0,Courtois,Real Madrid,Goalkeeper,1230,13,0,0,64.2
1,Vinícius Júnior,Real Madrid,Forward,1199,13,4,6,133.0
2,Benzema,Real Madrid,Forward,1106,12,15,1,121.5
3,Modrić,Real Madrid,Midfielder,1077,13,0,4,124.5
4,Éder Militão,Real Madrid,Defender,1076,12,0,0,110.4
...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0,0,0.7
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0,0,0.7
744,Cojocari,Sheriff,Defender,1,1,0,0,0.5
745,Maouassa,Club Brugge,Defender,1,1,0,0,0.2


In [5]:
disc_UCL = pd.read_csv('../Data/disciplinary.csv')
disc_UCL

,serial,player_name,club,position,fouls_committed,fouls_suffered,red,yellow,minutes_played,match_played
0,1,João Palhinha,Sporting CP,Midfielder,23,17,2,0,534,6
1,2,Capoue,Villarreal,Midfielder,19,17,3,1,1046,12
2,2,Rodri,Man. City,Midfielder,19,4,2,0,842,10
3,2,Seiwald,Salzburg,Midfielder,19,3,2,0,649,8
4,5,Balanta,Club Brugge,Midfielder,18,3,4,0,407,5
...,...,...,...,...,...,...,...,...,...,...
579,491,Eric Ramirez,Dynamo Kyiv,Forward,1,0,0,0,16,2
580,491,Bradarić,LOSC,Defender,1,0,0,0,13,1
581,491,Iborra,Villarreal,Midfielder,1,0,0,0,10,1
582,491,Zhegrova,LOSC,Midfielder,1,0,0,0,9,1


In [ ]:
# 0. (Optional) clean any old fouls columns from previous attempts
cols_to_drop = [c for c in new_mega_UCL.columns if c.startswith('fouls_committed')]
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_drop)

# 1. Take the fouls column from disciplinary df
disc_fouls = disc_UCL[['player_name', 'club', 'fouls_committed']]

# 2. Merge
new_mega_UCL = new_mega_UCL.merge(
    disc_fouls,
    on=['player_name', 'club'],
    how='left'
)

# 3. Fill missing with 0
new_mega_UCL['fouls_committed'] = (
    new_mega_UCL['fouls_committed']
    .fillna(0)
    .astype(int)
)
new_mega_UCL  #merges fouls_committed to new_mega_ucl

,player_name,club,position,minutes_played,match_played,goals,assists,distance_covered,fouls_committed
0,Courtois,Real Madrid,Goalkeeper,1230,13,0,0,64.2,0
1,Vinícius Júnior,Real Madrid,Forward,1199,13,4,6,133.0,13
2,Benzema,Real Madrid,Forward,1106,12,15,1,121.5,2
3,Modrić,Real Madrid,Midfielder,1077,13,0,4,124.5,3
4,Éder Militão,Real Madrid,Defender,1076,12,0,0,110.4,17
...,...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0,0,0.7,0
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0,0,0.7,0
744,Cojocari,Sheriff,Defender,1,1,0,0,0.5,0
745,Maouassa,Club Brugge,Defender,1,1,0,0,0.2,0


In [ ]:
new_mega_UCL.corr(numeric_only=True)['fouls_committed'].sort_values(ascending=False)  #correlation is fine

fouls_committed    1.000000
match_played       0.578350
minutes_played     0.547913
assists            0.258530
goals              0.220490
Name: fouls_committed, dtype: float64

In [ ]:
# Only create binary column once
if 'position_binary' not in new_mega_UCL.columns:
    new_mega_UCL['position_binary'] = new_mega_UCL['position'].map({
        'Defender': 0,
        'Forward': 1,
        'Goalkeeper': 0,
        'Midfielder': 1
    })

# X: all numeric features except text + target
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Y: the binary target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results) #fouls committed has p value below 0.05 so keep (statistically significant)

,coef,std err,z,P>|z|
minutes_played,-0.0369,0.004,-9.357,0.000
match_played,0.2376,0.068,3.514,0.000
goals,1.3904,0.208,6.677,0.000
assists,0.1915,0.145,1.323,0.186
distance_covered,0.2685,0.034,7.795,0.000
fouls_committed,0.0796,0.037,2.122,0.034


In [ ]:
# Remove any old copies if you tested before
cols_to_drop = [c for c in new_mega_UCL.columns if c.startswith('fouls_suffered')]
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_drop, errors='ignore')

# Extract only needed column
disc_foulssuf = disc_UCL[['player_name', 'club', 'fouls_suffered']]

# Merge into new_mega_UCL
new_mega_UCL = new_mega_UCL.merge(
    disc_foulssuf,
    on=['player_name', 'club'],
    how='left'
)

# Fill players with no fouls suffered
new_mega_UCL['fouls_suffered'] = new_mega_UCL['fouls_suffered'].fillna(0).astype(int)

new_mega_UCL #merges fouls suffered to new_mega_UCL


,player_name,club,position,minutes_played,match_played,goals,assists,distance_covered,fouls_committed,position_binary,fouls_suffered
0,Courtois,Real Madrid,Goalkeeper,1230,13,0,0,64.2,0,0,0
1,Vinícius Júnior,Real Madrid,Forward,1199,13,4,6,133.0,13,1,24
2,Benzema,Real Madrid,Forward,1106,12,15,1,121.5,2,1,4
3,Modrić,Real Madrid,Midfielder,1077,13,0,4,124.5,3,1,8
4,Éder Militão,Real Madrid,Defender,1076,12,0,0,110.4,17,0,16
...,...,...,...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0,0,0.7,0,1,0
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0,0,0.7,0,1,0
744,Cojocari,Sheriff,Defender,1,1,0,0,0.5,0,0,0
745,Maouassa,Club Brugge,Defender,1,1,0,0,0.2,0,0,0


In [ ]:
new_mega_UCL.corr(numeric_only=True)['fouls_suffered'].sort_values(ascending=False) #correlation is fine


fouls_suffered     1.000000
fouls_committed    0.585021
match_played       0.559968
minutes_played     0.551247
assists            0.370640
goals              0.332107
position_binary    0.251948
Name: fouls_suffered, dtype: float64

In [16]:
# Only create binary column once (safe, won't recreate)
if 'position_binary' not in new_mega_UCL.columns:
    new_mega_UCL['position_binary'] = new_mega_UCL['position'].map({
        'Defender': 0,
        'Forward': 1,
        'Goalkeeper': 0,
        'Midfielder': 1
    })

# X features (drop text + target)
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Y target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)



,coef,std err,z,P>|z|
minutes_played,-0.0377,0.004,-9.069,0.000
match_played,0.2445,0.069,3.535,0.000
goals,1.3323,0.228,5.836,0.000
assists,0.1071,0.156,0.687,0.492
distance_covered,0.2580,0.036,7.201,0.000
fouls_committed,0.0363,0.041,0.890,0.373
fouls_suffered,0.2370,0.043,5.533,0.000


In [ ]:
# dropping assists and fouls committed since statsitically insignificant with p value more than 0.05 (assists was statistically insignifcant already but forgot to drop earlier)
new_mega_UCL = new_mega_UCL.drop(columns=['fouls_committed'], errors='ignore')
new_mega_UCL = new_mega_UCL.drop(columns=['assists'], errors='ignore')

new_mega_UCL

,player_name,club,position,minutes_played,match_played,goals,distance_covered,position_binary,fouls_suffered
0,Courtois,Real Madrid,Goalkeeper,1230,13,0,64.2,0,0
1,Vinícius Júnior,Real Madrid,Forward,1199,13,4,133.0,1,24
2,Benzema,Real Madrid,Forward,1106,12,15,121.5,1,4
3,Modrić,Real Madrid,Midfielder,1077,13,0,124.5,1,8
4,Éder Militão,Real Madrid,Defender,1076,12,0,110.4,0,16
...,...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0,0.7,1,0
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0,0.7,1,0
744,Cojocari,Sheriff,Defender,1,1,0,0.5,0,0
745,Maouassa,Club Brugge,Defender,1,1,0,0.2,0,0


In [ ]:
# Remove old versions if any exist
cols_to_drop = [c for c in new_mega_UCL.columns if c.startswith('yellow')]
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_drop, errors='ignore')

# Extract yellow column from disciplinary df
disc_yellow = disc_UCL[['player_name', 'club', 'yellow']]

# Merge into main dataframe
new_mega_UCL = new_mega_UCL.merge(
    disc_yellow,
    on=['player_name', 'club'],
    how='left'
)

# Fill missing values (means player had 0 yellow cards)
new_mega_UCL['yellow'] = new_mega_UCL['yellow'].fillna(0).astype(int)

new_mega_UCL #merges yellow into new_mega_UCL


,player_name,club,position,minutes_played,match_played,goals,distance_covered,position_binary,fouls_suffered,yellow
0,Courtois,Real Madrid,Goalkeeper,1230,13,0,64.2,0,0,0
1,Vinícius Júnior,Real Madrid,Forward,1199,13,4,133.0,1,24,0
2,Benzema,Real Madrid,Forward,1106,12,15,121.5,1,4,0
3,Modrić,Real Madrid,Midfielder,1077,13,0,124.5,1,8,0
4,Éder Militão,Real Madrid,Defender,1076,12,0,110.4,0,16,0
...,...,...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0,0.7,1,0,0
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0,0.7,1,0,0
744,Cojocari,Sheriff,Defender,1,1,0,0.5,0,0,0
745,Maouassa,Club Brugge,Defender,1,1,0,0.2,0,0,0


In [ ]:
new_mega_UCL.corr(numeric_only=True)['yellow'].sort_values(ascending=False) #correlation is fine and even negative for position binary showing defenders (binary value = 0) are more likely to get yellow cards which makes sense


yellow             1.000000
minutes_played     0.045958
match_played       0.036185
goals             -0.012622
fouls_suffered    -0.016502
position_binary   -0.055708
Name: yellow, dtype: float64

In [21]:
# Only create binary position column once
if 'position_binary' not in new_mega_UCL.columns:
    new_mega_UCL['position_binary'] = new_mega_UCL['position'].map({
        'Defender': 0,
        'Forward': 1,
        'Goalkeeper': 0,
        'Midfielder': 1
    })

# X = numeric features (drop non-numeric + target)
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Y = binary target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)



,coef,std err,z,P>|z|
minutes_played,-0.0379,0.004,-9.104,0.000
match_played,0.2490,0.069,3.602,0.000
goals,1.3498,0.228,5.922,0.000
distance_covered,0.2630,0.036,7.391,0.000
fouls_suffered,0.2439,0.042,5.764,0.000
yellow,-0.1653,0.553,-0.299,0.765


In [ ]:
new_mega_UCL = new_mega_UCL.drop(columns=['yellow'], errors='ignore') #yellow is statistically insignificant with p value more than 0.05 so delete


In [ ]:
# Remove old versions if you tested before
cols_to_drop = [c for c in new_mega_UCL.columns if c.startswith('red')]
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_drop, errors='ignore')

# Extract red column from disciplinary df
disc_red = disc_UCL[['player_name', 'club', 'red']]

# Merge
new_mega_UCL = new_mega_UCL.merge(
    disc_red,
    on=['player_name', 'club'],
    how='left'
)

# Fill missing with 0
new_mega_UCL['red'] = new_mega_UCL['red'].fillna(0).astype(int)

new_mega_UCL #merges red into new_mega_UCL


,player_name,club,position,minutes_played,match_played,goals,distance_covered,position_binary,fouls_suffered,red
0,Courtois,Real Madrid,Goalkeeper,1230,13,0,64.2,0,0,0
1,Vinícius Júnior,Real Madrid,Forward,1199,13,4,133.0,1,24,1
2,Benzema,Real Madrid,Forward,1106,12,15,121.5,1,4,1
3,Modrić,Real Madrid,Midfielder,1077,13,0,124.5,1,8,1
4,Éder Militão,Real Madrid,Defender,1076,12,0,110.4,0,16,4
...,...,...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0,0.7,1,0,0
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0,0.7,1,0,0
744,Cojocari,Sheriff,Defender,1,1,0,0.5,0,0,0
745,Maouassa,Club Brugge,Defender,1,1,0,0.2,0,0,0


In [ ]:
new_mega_UCL.corr(numeric_only=True)['red'].sort_values(ascending=False) #correlation is fine


red                1.000000
fouls_suffered     0.393459
minutes_played     0.372903
match_played       0.366819
goals              0.048039
position_binary    0.033545
Name: red, dtype: float64

In [25]:
# Only create position_binary once
if 'position_binary' not in new_mega_UCL.columns:
    new_mega_UCL['position_binary'] = new_mega_UCL['position'].map({
        'Defender': 0,
        'Forward': 1,
        'Goalkeeper': 0,
        'Midfielder': 1
    })

# X features
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Y target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)




,coef,std err,z,P>|z|
minutes_played,-0.0382,0.004,-9.159,0.000
match_played,0.2452,0.069,3.543,0.000
goals,1.3629,0.227,5.998,0.000
distance_covered,0.2639,0.036,7.410,0.000
fouls_suffered,0.2410,0.042,5.685,0.000
red,0.1236,0.128,0.964,0.335


In [ ]:
new_mega_UCL = new_mega_UCL.drop(columns=['red'], errors='ignore') #red is statistically insignificant with p value more than 0.05 so delete


In [27]:
attempts_UCL = pd.read_csv('../Data/attempts.csv')
attempts_UCL

,serial,player_name,club,position,total_attempts,on_target,off_target,blocked,match_played
0,1,Benzema,Real Madrid,Forward,45,23,13,9,12
1,2,Salah,Liverpool,Forward,42,22,11,9,13
2,3,Mahrez,Man. City,Midfielder,37,18,8,11,12
3,4,Sané,Bayern,Midfielder,33,13,12,8,10
4,5,Lewandowski,Bayern,Forward,31,19,8,4,10
...,...,...,...,...,...,...,...,...,...
541,440,Dina Ebimbe,Paris,Midfielder,1,0,0,1,1
542,440,N'Koudou,Beşiktaş,Midfielder,1,0,0,1,1
543,440,Izquierdo,Club Brugge,Forward,1,0,0,1,1
544,440,Radonjić,Benfica,Forward,1,0,0,1,1


In [ ]:
# 1. Remove any old versions in case you re-ran cells earlier
cols_to_drop = [c for c in new_mega_UCL.columns if c.startswith('total_attempts')]
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_drop, errors='ignore')

# 2. Extract the needed column from attempts_UCL
attempts_total = attempts_UCL[['player_name', 'club', 'total_attempts']]

# 3. Merge into main dataframe
new_mega_UCL = new_mega_UCL.merge(
    attempts_total,
    on=['player_name', 'club'],
    how='left'
)

# 4. Fill missing values with 0 (players with no attempt data)
new_mega_UCL['total_attempts'] = new_mega_UCL['total_attempts'].fillna(0).astype(int)

new_mega_UCL #merges total attempts to new_mega_ucl


,player_name,club,position,minutes_played,match_played,goals,distance_covered,position_binary,fouls_suffered,total_attempts
0,Courtois,Real Madrid,Goalkeeper,1230,13,0,64.2,0,0,0
1,Vinícius Júnior,Real Madrid,Forward,1199,13,4,133.0,1,24,27
2,Benzema,Real Madrid,Forward,1106,12,15,121.5,1,4,45
3,Modrić,Real Madrid,Midfielder,1077,13,0,124.5,1,8,14
4,Éder Militão,Real Madrid,Defender,1076,12,0,110.4,0,16,9
...,...,...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0,0.7,1,0,0
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0,0.7,1,0,0
744,Cojocari,Sheriff,Defender,1,1,0,0.5,0,0,0
745,Maouassa,Club Brugge,Defender,1,1,0,0.2,0,0,0


In [ ]:
new_mega_UCL.corr(numeric_only=True)['total_attempts'].sort_values(ascending=False) #high correlation with goals


total_attempts     1.000000
goals              0.765944
fouls_suffered     0.540556
match_played       0.537510
minutes_played     0.498447
position_binary    0.317618
Name: total_attempts, dtype: float64

In [ ]:
# Only create binary target once (safe)
if 'position_binary' not in new_mega_UCL.columns:
    new_mega_UCL['position_binary'] = new_mega_UCL['position'].map({
        'Defender': 0,
        'Forward': 1,
        'Goalkeeper': 0,
        'Midfielder': 1
    })

# X = all numeric predictors except identifiers + target
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Y = target (attackers vs non-attackers)
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)

#despite high correlation, 0 p value showing it statstically signicant and captures somethings goals does not, so keeping attempts

,coef,std err,z,P>|z|
minutes_played,-0.0395,0.004,-8.893,0.000
match_played,0.1711,0.070,2.429,0.015
goals,0.7720,0.256,3.017,0.003
distance_covered,0.2652,0.038,7.022,0.000
fouls_suffered,0.2165,0.045,4.858,0.000
total_attempts,0.3178,0.051,6.207,0.000


In [ ]:
# 1. Remove any old versions if they exist
cols_to_drop = [c for c in new_mega_UCL.columns if c.startswith('on_target')]
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_drop, errors='ignore')

# 2. Extract on_target column from attempts_UCL
attempts_on_target = attempts_UCL[['player_name', 'club', 'on_target']]

# 3. Merge
new_mega_UCL = new_mega_UCL.merge(
    attempts_on_target,
    on=['player_name', 'club'],
    how='left'
)

# 4. Fill missing values with 0
new_mega_UCL['on_target'] = new_mega_UCL['on_target'].fillna(0).astype(int)

new_mega_UCL # merges on target to new_mega_UCL


,player_name,club,position,minutes_played,match_played,goals,distance_covered,position_binary,fouls_suffered,total_attempts,on_target
0,Courtois,Real Madrid,Goalkeeper,1230,13,0,64.2,0,0,0,0
1,Vinícius Júnior,Real Madrid,Forward,1199,13,4,133.0,1,24,27,8
2,Benzema,Real Madrid,Forward,1106,12,15,121.5,1,4,45,23
3,Modrić,Real Madrid,Midfielder,1077,13,0,124.5,1,8,14,5
4,Éder Militão,Real Madrid,Defender,1076,12,0,110.4,0,16,9,4
...,...,...,...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0,0.7,1,0,0,0
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0,0.7,1,0,0,0
744,Cojocari,Sheriff,Defender,1,1,0,0.5,0,0,0,0
745,Maouassa,Club Brugge,Defender,1,1,0,0.2,0,0,0,0


In [ ]:
new_mega_UCL.corr(numeric_only=True)['on_target'].sort_values(ascending=False) #very highly correlated with total attempts and goals


on_target          1.000000
total_attempts     0.906830
goals              0.869603
match_played       0.450144
fouls_suffered     0.443561
minutes_played     0.409441
position_binary    0.295246
Name: on_target, dtype: float64

In [35]:
# Only create position_binary once (safe)
if 'position_binary' not in new_mega_UCL.columns:
    new_mega_UCL['position_binary'] = new_mega_UCL['position'].map({
        'Defender': 0,
        'Forward': 1,
        'Goalkeeper': 0,
        'Midfielder': 1
    })

# X = numeric predictors
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Y = binary target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)



,coef,std err,z,P>|z|
minutes_played,-0.0395,0.004,-8.895,0.000
match_played,0.1678,0.070,2.385,0.017
goals,0.5974,0.285,2.094,0.036
distance_covered,0.2666,0.038,7.053,0.000
fouls_suffered,0.2202,0.045,4.925,0.000
total_attempts,0.2597,0.064,4.041,0.000
on_target,0.2343,0.164,1.430,0.153


In [ ]:
new_mega_UCL = new_mega_UCL.drop(columns=['on_target'], errors='ignore') #extreme correlation and high p value so delete on target


In [ ]:
# 1. Remove old versions if any exist
cols_to_drop = [c for c in new_mega_UCL.columns if c.startswith('off_target')]
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_drop, errors='ignore')

# 2. Extract the column from attempts_UCL
attempts_off_target = attempts_UCL[['player_name', 'club', 'off_target']]

# 3. Merge into main dataframe
new_mega_UCL = new_mega_UCL.merge(
    attempts_off_target,
    on=['player_name', 'club'],
    how='left'
)

# 4. Fill missing with 0
new_mega_UCL['off_target'] = new_mega_UCL['off_target'].fillna(0).astype(int)

new_mega_UCL #merges off target into new_mega_ucl


,player_name,club,position,minutes_played,match_played,goals,distance_covered,position_binary,fouls_suffered,total_attempts,off_target
0,Courtois,Real Madrid,Goalkeeper,1230,13,0,64.2,0,0,0,0
1,Vinícius Júnior,Real Madrid,Forward,1199,13,4,133.0,1,24,27,10
2,Benzema,Real Madrid,Forward,1106,12,15,121.5,1,4,45,13
3,Modrić,Real Madrid,Midfielder,1077,13,0,124.5,1,8,14,3
4,Éder Militão,Real Madrid,Defender,1076,12,0,110.4,0,16,9,5
...,...,...,...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0,0.7,1,0,0,0
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0,0.7,1,0,0,0
744,Cojocari,Sheriff,Defender,1,1,0,0.5,0,0,0,0
745,Maouassa,Club Brugge,Defender,1,1,0,0.2,0,0,0,0


In [38]:
new_mega_UCL.corr(numeric_only=True)['off_target'].sort_values(ascending=False)


off_target         1.000000
total_attempts     0.866831
goals              0.551769
fouls_suffered     0.510947
match_played       0.494840
minutes_played     0.471545
position_binary    0.241476
Name: off_target, dtype: float64

In [ ]:
# Only create the binary target once
if 'position_binary' not in new_mega_UCL.columns:
    new_mega_UCL['position_binary'] = new_mega_UCL['position'].map({
        'Defender': 0,
        'Forward': 1,
        'Goalkeeper': 0,
        'Midfielder': 1
    })

# X predictors
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Y target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)

# keeping off target because despite high correlation p value is below 0.05. also has negative coeffeicnt showing defenders are more likely to shoot off target which makes sense

,coef,std err,z,P>|z|
minutes_played,-0.0388,0.004,-8.685,0.000
match_played,0.1730,0.071,2.450,0.014
goals,0.6310,0.264,2.388,0.017
distance_covered,0.2595,0.038,6.816,0.000
fouls_suffered,0.2271,0.045,5.020,0.000
total_attempts,0.4747,0.085,5.612,0.000
off_target,-0.3264,0.132,-2.479,0.013


In [ ]:
# 1. Remove any previous versions
cols_to_drop = [c for c in new_mega_UCL.columns if c.startswith('blocked')]
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_drop, errors='ignore')

# 2. Extract blocked from attempts dataset
attempts_blocked = attempts_UCL[['player_name', 'club', 'blocked']]

# 3. Merge
new_mega_UCL = new_mega_UCL.merge(
    attempts_blocked,
    on=['player_name', 'club'],
    how='left'
)

# 4. Fill missing values with 0
new_mega_UCL['blocked'] = new_mega_UCL['blocked'].fillna(0).astype(int)

new_mega_UCL #merges blocked attempts into new_mega_UCL


,player_name,club,position,minutes_played,match_played,goals,distance_covered,position_binary,fouls_suffered,total_attempts,off_target,blocked
0,Courtois,Real Madrid,Goalkeeper,1230,13,0,64.2,0,0,0,0,0
1,Vinícius Júnior,Real Madrid,Forward,1199,13,4,133.0,1,24,27,10,9
2,Benzema,Real Madrid,Forward,1106,12,15,121.5,1,4,45,13,9
3,Modrić,Real Madrid,Midfielder,1077,13,0,124.5,1,8,14,3,6
4,Éder Militão,Real Madrid,Defender,1076,12,0,110.4,0,16,9,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0,0.7,1,0,0,0,0
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0,0.7,1,0,0,0,0
744,Cojocari,Sheriff,Defender,1,1,0,0.5,0,0,0,0,0
745,Maouassa,Club Brugge,Defender,1,1,0,0.2,0,0,0,0,0


In [ ]:
new_mega_UCL.corr(numeric_only=True)['blocked'].sort_values(ascending=False) #high correlation with total attempts which makes sense


blocked            1.000000
total_attempts     0.831373
off_target         0.608215
goals              0.508359
match_played       0.467747
fouls_suffered     0.467458
minutes_played     0.429849
position_binary    0.296571
Name: blocked, dtype: float64

In [44]:
# Only create binary target once
if 'position_binary' not in new_mega_UCL.columns:
    new_mega_UCL['position_binary'] = new_mega_UCL['position'].map({
        'Defender': 0,
        'Forward': 1,
        'Goalkeeper': 0,
        'Midfielder': 1
    })

# X predictors (numeric only)
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Y target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)



,coef,std err,z,P>|z|
minutes_played,-0.0389,0.004,-8.677,0.000
match_played,0.1727,0.071,2.445,0.015
goals,0.6218,0.285,2.179,0.029
distance_covered,0.2597,0.038,6.807,0.000
fouls_suffered,0.2271,0.045,5.021,0.000
total_attempts,0.4837,0.136,3.554,0.000
off_target,-0.3357,0.171,-1.963,0.050
blocked,-0.0169,0.198,-0.085,0.932


In [ ]:
new_mega_UCL = new_mega_UCL.drop(columns=['blocked'], errors='ignore') # drop blocked because high p value. keep off target because despite p value being 0.05, after dropping blocked it will be fine


In [47]:
attacking_UCL = pd.read_csv('../Data/attacking.csv')
attacking_UCL

,serial,player_name,club,position,assists,corner_taken,offsides,dribbles,match_played
0,1,Bruno Fernandes,Man. United,Midfielder,7,10,2,7,7
1,2,Vinícius Júnior,Real Madrid,Forward,6,3,4,83,13
2,2,Sané,Bayern,Midfielder,6,3,3,32,10
3,4,Antony,Ajax,Forward,5,3,4,28,7
4,5,Alexander-Arnold,Liverpool,Defender,4,36,0,9,9
...,...,...,...,...,...,...,...,...,...
171,64,Tolisso,Bayern,Midfielder,1,0,0,0,4
172,64,Schuurs,Ajax,Defender,1,0,0,0,3
173,64,Kryvtsov,Shakhtar Donetsk,Defender,1,0,0,0,3
174,64,Stanišić,Bayern,Defender,1,0,0,0,2


In [67]:
defending_UCL = pd.read_csv('../Data/defending.csv')
defending_UCL = defending_UCL.rename(columns={'balls_recoverd': 'balls_recovered'})
defending_UCL

,serial,player_name,club,position,balls_recovered,tackles,t_won,t_lost,clearance_attempted,match_played
0,1,Casemiro,Real Madrid,Midfielder,76,31,13,18,19,11
1,1,Éder Militão,Real Madrid,Defender,76,19,6,13,32,12
2,1,Reinildo,Atlético,Defender,76,13,6,7,20,10
3,4,Fabinho,Liverpool,Midfielder,74,23,9,14,8,13
4,5,Albiol,Villarreal,Defender,73,9,6,3,62,12
...,...,...,...,...,...,...,...,...,...,...
622,578,Maier,Young Boys,Midfielder,1,0,0,0,0,1
623,578,Zhegrova,LOSC,Midfielder,1,0,0,0,0,1
624,578,Zesiger,Young Boys,Defender,1,0,0,0,0,1
625,578,Zakaria,Juventus,Midfielder,1,0,0,0,0,1


In [ ]:
# 1. Remove any previous versions to avoid duplicates
cols_to_drop = [c for c in new_mega_UCL.columns if c.startswith('corner_taken')]
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_drop, errors='ignore')

# 2. Extract column from attacking_UCL
attacking_corner = attacking_UCL[['player_name', 'club', 'corner_taken']]

# 3. Merge into the main dataset
new_mega_UCL = new_mega_UCL.merge(
    attacking_corner,
    on=['player_name', 'club'],
    how='left'
)

# 4. Replace missing values (means 0 corners taken)
new_mega_UCL['corner_taken'] = new_mega_UCL['corner_taken'].fillna(0).astype(int)

new_mega_UCL #merges corner taken into new_mega_ucl


,player_name,club,position,minutes_played,match_played,goals,distance_covered,position_binary,fouls_suffered,total_attempts,off_target,corner_taken
0,Courtois,Real Madrid,Goalkeeper,1230,13,0,64.2,0,0,0,0,0
1,Vinícius Júnior,Real Madrid,Forward,1199,13,4,133.0,1,24,27,10,3
2,Benzema,Real Madrid,Forward,1106,12,15,121.5,1,4,45,13,2
3,Modrić,Real Madrid,Midfielder,1077,13,0,124.5,1,8,14,3,10
4,Éder Militão,Real Madrid,Defender,1076,12,0,110.4,0,16,9,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0,0.7,1,0,0,0,0
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0,0.7,1,0,0,0,0
744,Cojocari,Sheriff,Defender,1,1,0,0.5,0,0,0,0,0
745,Maouassa,Club Brugge,Defender,1,1,0,0.2,0,0,0,0,0


In [ ]:
new_mega_UCL.corr(numeric_only=True)['corner_taken'].sort_values(ascending=False) #no crazy correlation


corner_taken       1.000000
total_attempts     0.295472
minutes_played     0.260664
match_played       0.254979
fouls_suffered     0.237615
off_target         0.216028
goals              0.133541
position_binary    0.097908
Name: corner_taken, dtype: float64

In [51]:
# Only create binary target once
if 'position_binary' not in new_mega_UCL.columns:
    new_mega_UCL['position_binary'] = new_mega_UCL['position'].map({
        'Defender': 0,
        'Forward': 1,
        'Goalkeeper': 0,
        'Midfielder': 1
    })

# X features
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Y target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)



,coef,std err,z,P>|z|
minutes_played,-0.0389,0.004,-8.694,0.000
match_played,0.1784,0.071,2.505,0.012
goals,0.6432,0.265,2.423,0.015
distance_covered,0.2593,0.038,6.811,0.000
fouls_suffered,0.2254,0.045,4.992,0.000
total_attempts,0.4691,0.085,5.487,0.000
off_target,-0.3205,0.132,-2.421,0.015
corner_taken,0.0233,0.037,0.638,0.524


In [ ]:
new_mega_UCL = new_mega_UCL.drop(columns=['corner_taken'], errors='ignore') #high p value so drop corner taken


In [ ]:
# 1. Remove any previous versions
cols_to_drop = [c for c in new_mega_UCL.columns if c.startswith('offsides')]
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_drop, errors='ignore')

# 2. Extract offsides from attacking_UCL
attacking_offsides = attacking_UCL[['player_name', 'club', 'offsides']]

# 3. Merge into main df
new_mega_UCL = new_mega_UCL.merge(
    attacking_offsides,
    on=['player_name', 'club'],
    how='left'
)

# 4. Replace missing values with 0
new_mega_UCL['offsides'] = new_mega_UCL['offsides'].fillna(0).astype(int)

new_mega_UCL #merges offsides into new_mega_UCL


,player_name,club,position,minutes_played,match_played,goals,distance_covered,position_binary,fouls_suffered,total_attempts,off_target,offsides
0,Courtois,Real Madrid,Goalkeeper,1230,13,0,64.2,0,0,0,0,0
1,Vinícius Júnior,Real Madrid,Forward,1199,13,4,133.0,1,24,27,10,4
2,Benzema,Real Madrid,Forward,1106,12,15,121.5,1,4,45,13,9
3,Modrić,Real Madrid,Midfielder,1077,13,0,124.5,1,8,14,3,0
4,Éder Militão,Real Madrid,Defender,1076,12,0,110.4,0,16,9,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0,0.7,1,0,0,0,0
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0,0.7,1,0,0,0,0
744,Cojocari,Sheriff,Defender,1,1,0,0.5,0,0,0,0,0
745,Maouassa,Club Brugge,Defender,1,1,0,0.2,0,0,0,0,0


In [ ]:
new_mega_UCL.corr(numeric_only=True)['offsides'].sort_values(ascending=False) #correlation makes sense since higher with attackers but nothing crazy


offsides           1.000000
total_attempts     0.583942
goals              0.554547
off_target         0.409503
fouls_suffered     0.326493
match_played       0.323741
minutes_played     0.313183
position_binary    0.159453
Name: offsides, dtype: float64

In [59]:
# Only create binary target once (safe)
if 'position_binary' not in new_mega_UCL.columns:
    new_mega_UCL['position_binary'] = new_mega_UCL['position'].map({
        'Defender': 0,
        'Forward': 1,
        'Goalkeeper': 0,
        'Midfielder': 1
    })

# X features (drop identifiers + target)
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Y target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)



,coef,std err,z,P>|z|
minutes_played,-0.0389,0.004,-8.698,0.000
match_played,0.1708,0.071,2.417,0.016
goals,0.6425,0.264,2.433,0.015
distance_covered,0.2607,0.038,6.835,0.000
fouls_suffered,0.2272,0.045,5.034,0.000
total_attempts,0.4821,0.085,5.653,0.000
off_target,-0.3342,0.132,-2.529,0.011
offsides,-0.1099,0.169,-0.649,0.516


In [ ]:
new_mega_UCL = new_mega_UCL.drop(columns=['offsides'], errors='ignore') # high p value so drop offsides. could be due to fullbacks being offisde often


In [ ]:
# 1. Remove old versions if they exist
cols_to_drop = [c for c in new_mega_UCL.columns if c.startswith('dribbles')]
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_drop, errors='ignore')

# 2. Extract dribbles from attacking dataset
attacking_dribbles = attacking_UCL[['player_name', 'club', 'dribbles']]

# 3. Merge into main df
new_mega_UCL = new_mega_UCL.merge(
    attacking_dribbles,
    on=['player_name', 'club'],
    how='left'
)

# 4. Players with no attacking data → dribbles = 0
new_mega_UCL['dribbles'] = new_mega_UCL['dribbles'].fillna(0).astype(int)

new_mega_UCL #merges dribbles into new_mega_UCL


,player_name,club,position,minutes_played,match_played,goals,distance_covered,position_binary,fouls_suffered,total_attempts,off_target,dribbles
0,Courtois,Real Madrid,Goalkeeper,1230,13,0,64.2,0,0,0,0,0
1,Vinícius Júnior,Real Madrid,Forward,1199,13,4,133.0,1,24,27,10,83
2,Benzema,Real Madrid,Forward,1106,12,15,121.5,1,4,45,13,18
3,Modrić,Real Madrid,Midfielder,1077,13,0,124.5,1,8,14,3,8
4,Éder Militão,Real Madrid,Defender,1076,12,0,110.4,0,16,9,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0,0.7,1,0,0,0,0
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0,0.7,1,0,0,0,0
744,Cojocari,Sheriff,Defender,1,1,0,0.5,0,0,0,0,0
745,Maouassa,Club Brugge,Defender,1,1,0,0.2,0,0,0,0,0


In [ ]:
new_mega_UCL.corr(numeric_only=True)['dribbles'].sort_values(ascending=False) #correlation is not crazy and makes sense


dribbles           1.000000
total_attempts     0.563173
off_target         0.457824
fouls_suffered     0.450734
goals              0.417832
match_played       0.364329
minutes_played     0.355881
position_binary    0.171731
Name: dribbles, dtype: float64

In [64]:
# Only create position_binary once
if 'position_binary' not in new_mega_UCL.columns:
    new_mega_UCL['position_binary'] = new_mega_UCL['position'].map({
        'Defender': 0,
        'Forward': 1,
        'Goalkeeper': 0,
        'Midfielder': 1
    })

# X = numeric predictors
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Y target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)



,coef,std err,z,P>|z|
minutes_played,-0.0390,0.004,-8.671,0.000
match_played,0.1703,0.071,2.403,0.016
goals,0.6520,0.269,2.422,0.015
distance_covered,0.2614,0.038,6.802,0.000
fouls_suffered,0.2274,0.045,5.046,0.000
total_attempts,0.4788,0.085,5.638,0.000
off_target,-0.3312,0.132,-2.508,0.012
dribbles,-0.0125,0.029,-0.426,0.670


In [ ]:
new_mega_UCL = new_mega_UCL.drop(columns=['dribbles'], errors='ignore') # high p value (0.67) so drop dribbles

In [ ]:
# 1. Remove any old versions
cols_to_drop = [c for c in new_mega_UCL.columns if c.startswith('balls_recovered')]
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_drop, errors='ignore')

# 2. Get column from defending dataset
defending_balls = defending_UCL[['player_name', 'club', 'balls_recovered']]

# 3. Merge into main DF
new_mega_UCL = new_mega_UCL.merge(
    defending_balls,
    on=['player_name', 'club'],
    how='left'
)

# 4. Fill missing = 0 (means no defensive data)
new_mega_UCL['balls_recovered'] = new_mega_UCL['balls_recovered'].fillna(0).astype(int)

new_mega_UCL #merges balls recovered into new_mega_ucl


,player_name,club,position,minutes_played,match_played,goals,distance_covered,position_binary,fouls_suffered,total_attempts,off_target,balls_recovered
0,Courtois,Real Madrid,Goalkeeper,1230,13,0,64.2,0,0,0,0,0
1,Vinícius Júnior,Real Madrid,Forward,1199,13,4,133.0,1,24,27,10,29
2,Benzema,Real Madrid,Forward,1106,12,15,121.5,1,4,45,13,6
3,Modrić,Real Madrid,Midfielder,1077,13,0,124.5,1,8,14,3,39
4,Éder Militão,Real Madrid,Defender,1076,12,0,110.4,0,16,9,5,76
...,...,...,...,...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0,0.7,1,0,0,0,0
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0,0.7,1,0,0,0,0
744,Cojocari,Sheriff,Defender,1,1,0,0.5,0,0,0,0,0
745,Maouassa,Club Brugge,Defender,1,1,0,0.2,0,0,0,0,0


In [74]:
new_mega_UCL.corr(numeric_only=True)['balls_recovered'].sort_values(ascending=False) #moderate correlation




balls_recovered    1.000000
minutes_played     0.670936
match_played       0.574400
fouls_suffered     0.394775
off_target         0.224677
total_attempts     0.158937
position_binary   -0.224962
Name: balls_recovered, dtype: float64

In [ ]:
# Only create the binary target once
if 'position_binary' not in new_mega_UCL.columns:
    new_mega_UCL['position_binary'] = new_mega_UCL['position'].map({
        'Defender': 0,
        'Forward': 1,
        'Goalkeeper': 0,
        'Midfielder': 1
    })

# X predictors
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Y target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)

# when adding balls recovered, it had p value of 0 but goals had high p value of 0.073 and match played had high p value of 0.069. I deleted goals and reran the model and balls recovered stayed at 0 and match played lowered to 0.046. Since we already have 
# total attempts and off target, which together capture almost all the information goals provide its ok to delete goals.


,coef,std err,z,P>|z|
minutes_played,-0.0338,0.004,-7.675,0.000
match_played,0.1430,0.071,2.000,0.046
distance_covered,0.2585,0.038,6.794,0.000
fouls_suffered,0.2146,0.046,4.690,0.000
total_attempts,0.4666,0.084,5.586,0.000
off_target,-0.3651,0.134,-2.722,0.006
balls_recovered,-0.0710,0.017,-4.211,0.000


In [ ]:
# 1. Remove previous versions if they exist
cols_to_drop = [c for c in new_mega_UCL.columns if c.startswith('tackles')]
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_drop, errors='ignore')

# 2. Extract tackles column from defending_UCL
defending_tackles = defending_UCL[['player_name', 'club', 'tackles']]

# 3. Merge
new_mega_UCL = new_mega_UCL.merge(
    defending_tackles,
    on=['player_name', 'club'],
    how='left'
)

# 4. Fill missing with 0
new_mega_UCL['tackles'] = new_mega_UCL['tackles'].fillna(0).astype(int)

new_mega_UCL #merges tackles into new_mega_UCL


,player_name,club,position,minutes_played,match_played,distance_covered,position_binary,fouls_suffered,total_attempts,off_target,balls_recovered,tackles
0,Courtois,Real Madrid,Goalkeeper,1230,13,64.2,0,0,0,0,0,0
1,Vinícius Júnior,Real Madrid,Forward,1199,13,133.0,1,24,27,10,29,11
2,Benzema,Real Madrid,Forward,1106,12,121.5,1,4,45,13,6,1
3,Modrić,Real Madrid,Midfielder,1077,13,124.5,1,8,14,3,39,9
4,Éder Militão,Real Madrid,Defender,1076,12,110.4,0,16,9,5,76,19
...,...,...,...,...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0.7,1,0,0,0,0,0
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0.7,1,0,0,0,0,0
744,Cojocari,Sheriff,Defender,1,1,0.5,0,0,0,0,0,0
745,Maouassa,Club Brugge,Defender,1,1,0.2,0,0,0,0,0,0


In [ ]:
new_mega_UCL.corr(numeric_only=True)['tackles'].sort_values(ascending=False) #moderate correlation a little higher with balls recovered which makes sense


tackles            1.000000
balls_recovered    0.693717
minutes_played     0.518764
match_played       0.495463
fouls_suffered     0.480082
off_target         0.206213
total_attempts     0.154207
position_binary    0.013443
Name: tackles, dtype: float64

In [ ]:
# Only create binary target once (safe)
if 'position_binary' not in new_mega_UCL.columns:
    new_mega_UCL['position_binary'] = new_mega_UCL['position'].map({
        'Defender': 0,
        'Forward': 1,
        'Goalkeeper': 0,
        'Midfielder': 1
    })

# X predictors
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Y target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)
# p values less than 0.05 so keep tackles. I am assuming this is more like an "attempted tackles" stat because the next 2 features being tested are "tackles won" and "tackles lost"

,coef,std err,z,P>|z|
minutes_played,-0.0325,0.004,-7.342,0.000
match_played,0.1415,0.072,1.966,0.049
distance_covered,0.2456,0.038,6.395,0.000
fouls_suffered,0.1908,0.047,4.073,0.000
total_attempts,0.4752,0.084,5.687,0.000
off_target,-0.3695,0.135,-2.728,0.006
balls_recovered,-0.0888,0.019,-4.670,0.000
tackles,0.0910,0.037,2.441,0.015


In [78]:
# 1. Remove old versions if they exist
cols_to_drop = [c for c in new_mega_UCL.columns if c.startswith('t_won')]
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_drop, errors='ignore')

# 2. Extract t_won from defending df
defending_twon = defending_UCL[['player_name', 'club', 't_won']]

# 3. Merge into main df
new_mega_UCL = new_mega_UCL.merge(
    defending_twon,
    on=['player_name', 'club'],
    how='left'
)

# 4. Fill missing with 0
new_mega_UCL['t_won'] = new_mega_UCL['t_won'].fillna(0).astype(int)

new_mega_UCL #merges tackles won into new_mega_UCL


,player_name,club,position,minutes_played,match_played,distance_covered,position_binary,fouls_suffered,total_attempts,off_target,balls_recovered,tackles,t_won
0,Courtois,Real Madrid,Goalkeeper,1230,13,64.2,0,0,0,0,0,0,0
1,Vinícius Júnior,Real Madrid,Forward,1199,13,133.0,1,24,27,10,29,11,3
2,Benzema,Real Madrid,Forward,1106,12,121.5,1,4,45,13,6,1,1
3,Modrić,Real Madrid,Midfielder,1077,13,124.5,1,8,14,3,39,9,3
4,Éder Militão,Real Madrid,Defender,1076,12,110.4,0,16,9,5,76,19,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0.7,1,0,0,0,0,0,0
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0.7,1,0,0,0,0,0,0
744,Cojocari,Sheriff,Defender,1,1,0.5,0,0,0,0,0,0,0
745,Maouassa,Club Brugge,Defender,1,1,0.2,0,0,0,0,0,0,0


In [ ]:
new_mega_UCL.corr(numeric_only=True)['t_won'].sort_values(ascending=False) #very high correlation with tackles which makes sense


t_won              1.000000
tackles            0.905772
balls_recovered    0.661969
minutes_played     0.454692
match_played       0.422240
fouls_suffered     0.380672
off_target         0.131604
total_attempts     0.073050
position_binary   -0.063866
Name: t_won, dtype: float64

In [ ]:
# X = numeric predictors
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Y = target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)

# despite very highly correlated with tackles, keep tackles won. This is because both have very low p values despite being highly correlated. Also could be explained by
# midfielders going for lots of tackles and losing them while defednders get less volume but win way more tackles.


,coef,std err,z,P>|z|
minutes_played,-0.0326,0.004,-7.334,0.000
match_played,0.1466,0.072,2.036,0.042
distance_covered,0.2433,0.038,6.366,0.000
fouls_suffered,0.1948,0.047,4.115,0.000
total_attempts,0.4772,0.085,5.620,0.000
off_target,-0.3629,0.138,-2.636,0.008
balls_recovered,-0.0838,0.019,-4.379,0.000
tackles,0.2031,0.061,3.306,0.001
t_won,-0.2585,0.108,-2.384,0.017


In [92]:
# 1. Remove any old versions if they exist
cols_to_drop = [c for c in new_mega_UCL.columns if c.startswith('t_lost')]
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_drop, errors='ignore')

# 2. Extract t_lost from defending dataset
defending_tlost = defending_UCL[['player_name', 'club', 't_lost']]

# 3. Merge
new_mega_UCL = new_mega_UCL.merge(
    defending_tlost,
    on=['player_name', 'club'],
    how='left'
)

# 4. Fill missing with 0 (means no defensive appearances)
new_mega_UCL['t_lost'] = new_mega_UCL['t_lost'].fillna(0).astype(int)

new_mega_UCL #merges tackles lost into new_mega_UCL


,player_name,club,position,minutes_played,match_played,distance_covered,position_binary,fouls_suffered,total_attempts,off_target,balls_recovered,clearance_attempted,t_lost
0,Courtois,Real Madrid,Goalkeeper,1230,13,64.2,0,0,0,0,0,0,0
1,Vinícius Júnior,Real Madrid,Forward,1199,13,133.0,1,24,27,10,29,0,8
2,Benzema,Real Madrid,Forward,1106,12,121.5,1,4,45,13,6,10,0
3,Modrić,Real Madrid,Midfielder,1077,13,124.5,1,8,14,3,39,4,6
4,Éder Militão,Real Madrid,Defender,1076,12,110.4,0,16,9,5,76,32,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0.7,1,0,0,0,0,0,0
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0.7,1,0,0,0,0,0,0
744,Cojocari,Sheriff,Defender,1,1,0.5,0,0,0,0,0,0,0
745,Maouassa,Club Brugge,Defender,1,1,0.2,0,0,0,0,0,0,0


In [93]:
new_mega_UCL.corr(numeric_only=True)['t_lost'].sort_values(ascending=False) #exteremely highly correlated with tackles 


t_lost                 1.000000
balls_recovered        0.613914
minutes_played         0.494626
fouls_suffered         0.493091
match_played           0.483007
clearance_attempted    0.348782
off_target             0.239918
total_attempts         0.201763
position_binary        0.079478
Name: t_lost, dtype: float64

In [94]:
# Predictors
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)
# since tackles lost and tackles are very highly correlated but tackles lost has way better p value, delete tackles.
# Then rerun model to see if tackles won p value imporves when deleting tackles

,coef,std err,z,P>|z|
minutes_played,-0.0256,0.004,-6.005,0.000
match_played,0.1390,0.075,1.860,0.063
distance_covered,0.2081,0.039,5.360,0.000
fouls_suffered,0.1440,0.051,2.815,0.005
total_attempts,0.4102,0.090,4.543,0.000
off_target,-0.3083,0.150,-2.060,0.039
balls_recovered,-0.0388,0.021,-1.847,0.065
clearance_attempted,-0.2439,0.042,-5.841,0.000
t_lost,0.1744,0.064,2.740,0.006


In [ ]:
new_mega_UCL = new_mega_UCL.drop(columns=['tackles'], errors='ignore') 


In [96]:
# Predictors
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)

,coef,std err,z,P>|z|
minutes_played,-0.0256,0.004,-6.005,0.000
match_played,0.1390,0.075,1.860,0.063
distance_covered,0.2081,0.039,5.360,0.000
fouls_suffered,0.1440,0.051,2.815,0.005
total_attempts,0.4102,0.090,4.543,0.000
off_target,-0.3083,0.150,-2.060,0.039
balls_recovered,-0.0388,0.021,-1.847,0.065
clearance_attempted,-0.2439,0.042,-5.841,0.000
t_lost,0.1744,0.064,2.740,0.006


In [97]:
new_mega_UCL = new_mega_UCL.drop(columns=['t_won'], errors='ignore') # p value increased so delete tackles won

In [98]:
# 1. Remove any old versions if they exist
cols_to_drop = [c for c in new_mega_UCL.columns if c.startswith('clearance_attempted')]
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_drop, errors='ignore')

# 2. Extract clearance_attempted from defending dataset
defending_clearance = defending_UCL[['player_name', 'club', 'clearance_attempted']]

# 3. Merge
new_mega_UCL = new_mega_UCL.merge(
    defending_clearance,
    on=['player_name', 'club'],
    how='left'
)

# 4. Fill missing with 0 (means no defensive appearances)
new_mega_UCL['clearance_attempted'] = new_mega_UCL['clearance_attempted'].fillna(0).astype(int)

new_mega_UCL 



,player_name,club,position,minutes_played,match_played,distance_covered,position_binary,fouls_suffered,total_attempts,off_target,balls_recovered,t_lost,clearance_attempted
0,Courtois,Real Madrid,Goalkeeper,1230,13,64.2,0,0,0,0,0,0,0
1,Vinícius Júnior,Real Madrid,Forward,1199,13,133.0,1,24,27,10,29,8,0
2,Benzema,Real Madrid,Forward,1106,12,121.5,1,4,45,13,6,0,10
3,Modrić,Real Madrid,Midfielder,1077,13,124.5,1,8,14,3,39,6,4
4,Éder Militão,Real Madrid,Defender,1076,12,110.4,0,16,9,5,76,13,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0.7,1,0,0,0,0,0,0
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0.7,1,0,0,0,0,0,0
744,Cojocari,Sheriff,Defender,1,1,0.5,0,0,0,0,0,0,0
745,Maouassa,Club Brugge,Defender,1,1,0.2,0,0,0,0,0,0,0


In [ ]:
new_mega_UCL.corr(numeric_only=True)['clearance_attempted'].sort_values(ascending=False) #moderate correlation higher with balls recovered


clearance_attempted    1.000000
balls_recovered        0.739776
minutes_played         0.496135
match_played           0.376168
t_lost                 0.348782
fouls_suffered         0.137384
off_target             0.089541
total_attempts         0.007276
position_binary       -0.408729
Name: clearance_attempted, dtype: float64

In [100]:
# Predictors
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)

,coef,std err,z,P>|z|
minutes_played,-0.0256,0.004,-6.005,0.000
match_played,0.1390,0.075,1.860,0.063
distance_covered,0.2081,0.039,5.360,0.000
fouls_suffered,0.1440,0.051,2.815,0.005
total_attempts,0.4102,0.090,4.543,0.000
off_target,-0.3083,0.150,-2.060,0.039
balls_recovered,-0.0388,0.021,-1.847,0.065
t_lost,0.1744,0.064,2.740,0.006
clearance_attempted,-0.2439,0.042,-5.841,0.000


In [101]:
distribution_UCL = pd.read_csv('../Data/distributon.csv')
distribution_UCL

,serial,player_name,club,position,pass_accuracy,pass_attempted,pass_completed,cross_accuracy,cross_attempted,cross_complted,freekicks_taken,match_played
0,1,Erokhin,Zenit,Midfielder,98.0,27,26,0,2,0,0,6
1,2,Eric García,Barcelona,Defender,97.5,203,197,0,0,0,1,4
2,3,Ulreich,Bayern,Goalkeeper,97.0,32,31,0,0,0,1,1
3,4,Rugani,Juventus,Defender,96.8,130,122,0,0,0,0,4
4,5,J. Timber,Ajax,Midfielder,95.8,497,474,0,1,0,14,8
...,...,...,...,...,...,...,...,...,...,...,...,...
603,604,Phillips,Liverpool,Defender,44.5,37,33,0,0,0,1,2
604,605,Philipp,Wolfsburg,Forward,44.0,9,7,0,0,0,1,2
605,606,Brobbey,Leipzig,Forward,42.6,23,13,0,2,0,0,5
606,607,Traoré,Shakhtar Donetsk,Forward,36.5,22,16,0,0,0,0,2


In [ ]:
# 1. Remove any old versions if they exist
cols_to_drop = [c for c in new_mega_UCL.columns if c.startswith('pass_accuracy')]
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_drop, errors='ignore')

# 2. Extract pass_accuracy from distribution dataset
distribution_passacc = distribution_UCL[['player_name', 'club', 'pass_accuracy']]

# 3. Merge
new_mega_UCL = new_mega_UCL.merge(
    distribution_passacc,
    on=['player_name', 'club'],
    how='left'
)

# 4. Fill missing with 0
new_mega_UCL['pass_accuracy'] = new_mega_UCL['pass_accuracy'].fillna(0)

new_mega_UCL #merges pass accuracy


,player_name,club,position,minutes_played,match_played,distance_covered,position_binary,fouls_suffered,total_attempts,off_target,balls_recovered,t_lost,clearance_attempted,pass_accuracy
0,Courtois,Real Madrid,Goalkeeper,1230,13,64.2,0,0,0,0,0,0,0,76.7
1,Vinícius Júnior,Real Madrid,Forward,1199,13,133.0,1,24,27,10,29,8,0,83.1
2,Benzema,Real Madrid,Forward,1106,12,121.5,1,4,45,13,6,0,10,83.1
3,Modrić,Real Madrid,Midfielder,1077,13,124.5,1,8,14,3,39,6,4,89.8
4,Éder Militão,Real Madrid,Defender,1076,12,110.4,0,16,9,5,76,13,32,87.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0.7,1,0,0,0,0,0,0,0.0
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0.7,1,0,0,0,0,0,0,0.0
744,Cojocari,Sheriff,Defender,1,1,0.5,0,0,0,0,0,0,0,0.0
745,Maouassa,Club Brugge,Defender,1,1,0.2,0,0,0,0,0,0,0,0.0


In [107]:
new_mega_UCL.corr(numeric_only=True)['pass_accuracy'].sort_values(ascending=False)


pass_accuracy          1.000000
minutes_played         0.596276
match_played           0.545774
balls_recovered        0.481060
fouls_suffered         0.368165
t_lost                 0.360490
clearance_attempted    0.328352
off_target             0.298750
total_attempts         0.276962
position_binary       -0.127762
Name: pass_accuracy, dtype: float64

In [110]:
# Predictors
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)


,coef,std err,z,P>|z|
minutes_played,-0.0246,0.004,-5.693,0.000
match_played,0.1719,0.082,2.106,0.035
distance_covered,0.2027,0.039,5.204,0.000
fouls_suffered,0.1415,0.051,2.776,0.006
total_attempts,0.4048,0.090,4.493,0.000
off_target,-0.2991,0.150,-2.000,0.046
balls_recovered,-0.0389,0.021,-1.853,0.064
t_lost,0.1724,0.064,2.711,0.007
clearance_attempted,-0.2471,0.042,-5.877,0.000
pass_accuracy,-0.0036,0.003,-1.042,0.297


In [111]:
new_mega_UCL = new_mega_UCL.drop(columns=['pass_accuracy'], errors='ignore') 

In [113]:
# 1. Remove any old versions if they exist
cols_to_drop = [c for c in new_mega_UCL.columns if c.startswith('pass_attempted')]
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_drop, errors='ignore')

# 2. Extract pass_attempted from distribution dataset
distribution_passattempt = distribution_UCL[['player_name', 'club', 'pass_attempted']]

# 3. Merge
new_mega_UCL = new_mega_UCL.merge(
    distribution_passattempt,
    on=['player_name', 'club'],
    how='left'
)

# 4. Fill missing with 0
new_mega_UCL['pass_attempted'] = new_mega_UCL['pass_attempted'].fillna(0)

new_mega_UCL


,player_name,club,position,minutes_played,match_played,distance_covered,position_binary,fouls_suffered,total_attempts,off_target,balls_recovered,t_lost,clearance_attempted,pass_attempted
0,Courtois,Real Madrid,Goalkeeper,1230,13,64.2,0,0,0,0,0,0,0,483.0
1,Vinícius Júnior,Real Madrid,Forward,1199,13,133.0,1,24,27,10,29,8,0,451.0
2,Benzema,Real Madrid,Forward,1106,12,121.5,1,4,45,13,6,0,10,523.0
3,Modrić,Real Madrid,Midfielder,1077,13,124.5,1,8,14,3,39,6,4,799.0
4,Éder Militão,Real Madrid,Defender,1076,12,110.4,0,16,9,5,76,13,32,643.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0.7,1,0,0,0,0,0,0,0.0
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0.7,1,0,0,0,0,0,0,0.0
744,Cojocari,Sheriff,Defender,1,1,0.5,0,0,0,0,0,0,0,0.0
745,Maouassa,Club Brugge,Defender,1,1,0.2,0,0,0,0,0,0,0,0.0


In [114]:
new_mega_UCL.corr(numeric_only=True)['pass_attempted'].sort_values(ascending=False)


pass_attempted         1.000000
minutes_played         0.860922
balls_recovered        0.787473
match_played           0.758473
t_lost                 0.505350
clearance_attempted    0.498611
fouls_suffered         0.439364
off_target             0.360392
total_attempts         0.355774
position_binary       -0.167090
Name: pass_attempted, dtype: float64

In [115]:
# Predictors
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)


,coef,std err,z,P>|z|
minutes_played,-0.0237,0.004,-5.472,0.000
match_played,0.1139,0.076,1.499,0.134
distance_covered,0.2165,0.039,5.498,0.000
fouls_suffered,0.1153,0.052,2.204,0.028
total_attempts,0.4131,0.091,4.530,0.000
off_target,-0.3198,0.152,-2.098,0.036
balls_recovered,-0.0134,0.023,-0.572,0.567
t_lost,0.1588,0.064,2.471,0.013
clearance_attempted,-0.2679,0.044,-6.156,0.000
pass_attempted,-0.0055,0.002,-2.283,0.022


In [116]:
# 1. Remove any old versions if they exist
cols_to_drop = [c for c in new_mega_UCL.columns if c.startswith('pass_completed')]
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_drop, errors='ignore')

# 2. Extract pass_completed from distribution dataset
distribution_passcompleted = distribution_UCL[['player_name', 'club', 'pass_completed']]

# 3. Merge
new_mega_UCL = new_mega_UCL.merge(
    distribution_passcompleted,
    on=['player_name', 'club'],
    how='left'
)

# 4. Fill missing with 0
new_mega_UCL['pass_completed'] = new_mega_UCL['pass_completed'].fillna(0)
new_mega_UCL

,player_name,club,position,minutes_played,match_played,distance_covered,position_binary,fouls_suffered,total_attempts,off_target,balls_recovered,t_lost,clearance_attempted,pass_attempted,pass_completed
0,Courtois,Real Madrid,Goalkeeper,1230,13,64.2,0,0,0,0,0,0,0,483.0,365.0
1,Vinícius Júnior,Real Madrid,Forward,1199,13,133.0,1,24,27,10,29,8,0,451.0,377.0
2,Benzema,Real Madrid,Forward,1106,12,121.5,1,4,45,13,6,0,10,523.0,433.0
3,Modrić,Real Madrid,Midfielder,1077,13,124.5,1,8,14,3,39,6,4,799.0,723.0
4,Éder Militão,Real Madrid,Defender,1076,12,110.4,0,16,9,5,76,13,32,643.0,563.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0.7,1,0,0,0,0,0,0,0.0,0.0
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0.7,1,0,0,0,0,0,0,0.0,0.0
744,Cojocari,Sheriff,Defender,1,1,0.5,0,0,0,0,0,0,0,0.0,0.0
745,Maouassa,Club Brugge,Defender,1,1,0.2,0,0,0,0,0,0,0,0.0,0.0


In [117]:
new_mega_UCL.corr(numeric_only=True)['pass_completed'].sort_values(ascending=False)


pass_completed         1.000000
pass_attempted         0.995174
minutes_played         0.827793
balls_recovered        0.790912
match_played           0.727849
t_lost                 0.500457
clearance_attempted    0.497273
fouls_suffered         0.421544
off_target             0.343284
total_attempts         0.332907
position_binary       -0.164570
Name: pass_completed, dtype: float64

In [118]:
# Predictors
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)


,coef,std err,z,P>|z|
minutes_played,-0.0220,0.004,-4.924,0.000
match_played,0.1088,0.077,1.417,0.157
distance_covered,0.2212,0.041,5.460,0.000
fouls_suffered,0.1143,0.053,2.171,0.030
total_attempts,0.4308,0.092,4.681,0.000
off_target,-0.3516,0.153,-2.299,0.021
balls_recovered,-0.0162,0.024,-0.684,0.494
t_lost,0.1479,0.065,2.276,0.023
clearance_attempted,-0.2632,0.044,-6.033,0.000
pass_attempted,-0.0398,0.015,-2.667,0.008


In [119]:
# features to delete
cols_to_delete = ['balls_recovered', 'pass_completed']

# drop them safely
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_delete, errors='ignore')
new_mega_UCL

,player_name,club,position,minutes_played,match_played,distance_covered,position_binary,fouls_suffered,total_attempts,off_target,t_lost,clearance_attempted,pass_attempted
0,Courtois,Real Madrid,Goalkeeper,1230,13,64.2,0,0,0,0,0,0,483.0
1,Vinícius Júnior,Real Madrid,Forward,1199,13,133.0,1,24,27,10,8,0,451.0
2,Benzema,Real Madrid,Forward,1106,12,121.5,1,4,45,13,0,10,523.0
3,Modrić,Real Madrid,Midfielder,1077,13,124.5,1,8,14,3,6,4,799.0
4,Éder Militão,Real Madrid,Defender,1076,12,110.4,0,16,9,5,13,32,643.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0.7,1,0,0,0,0,0,0.0
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0.7,1,0,0,0,0,0,0.0
744,Cojocari,Sheriff,Defender,1,1,0.5,0,0,0,0,0,0,0.0
745,Maouassa,Club Brugge,Defender,1,1,0.2,0,0,0,0,0,0,0.0


In [120]:
# 1. Remove any old versions if they exist
cols_to_drop = [c for c in new_mega_UCL.columns if c.startswith('cross_accuracy')]
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_drop, errors='ignore')

# 2. Extract cross_accuracy from distribution dataset
distribution_crossacc = distribution_UCL[['player_name', 'club', 'cross_accuracy']]

# 3. Merge
new_mega_UCL = new_mega_UCL.merge(
    distribution_crossacc,
    on=['player_name', 'club'],
    how='left'
)

# 4. Fill missing with 0
new_mega_UCL['cross_accuracy'] = new_mega_UCL['cross_accuracy'].fillna(0)


In [121]:
new_mega_UCL.corr(numeric_only=True)['cross_accuracy'].sort_values(ascending=False)


cross_accuracy         1.000000
fouls_suffered         0.285398
match_played           0.260998
total_attempts         0.248776
off_target             0.222071
minutes_played         0.221600
pass_attempted         0.216287
t_lost                 0.147599
position_binary        0.120460
clearance_attempted    0.007652
Name: cross_accuracy, dtype: float64

In [122]:
# Predictors
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)


,coef,std err,z,P>|z|
minutes_played,-0.0234,0.004,-5.434,0.000
match_played,0.1317,0.076,1.722,0.085
distance_covered,0.2131,0.039,5.476,0.000
fouls_suffered,0.1184,0.053,2.245,0.025
total_attempts,0.4264,0.091,4.666,0.000
off_target,-0.3320,0.152,-2.185,0.029
t_lost,0.1454,0.062,2.344,0.019
clearance_attempted,-0.2816,0.040,-7.076,0.000
pass_attempted,-0.0061,0.002,-2.796,0.005
cross_accuracy,-0.0066,0.006,-1.109,0.267


In [123]:
# delete cross_accuracy due to high p value
new_mega_UCL = new_mega_UCL.drop(columns=['cross_accuracy'], errors='ignore')


In [124]:
# 1. Remove any old versions if they exist
cols_to_drop = [c for c in new_mega_UCL.columns if c.startswith('cross_attempted')]
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_drop, errors='ignore')

# 2. Extract from distribution dataset
distribution_crossattempt = distribution_UCL[['player_name', 'club', 'cross_attempted']]

# 3. Merge
new_mega_UCL = new_mega_UCL.merge(
    distribution_crossattempt,
    on=['player_name', 'club'],
    how='left'
)

# 4. Fill missing with 0
new_mega_UCL['cross_attempted'] = new_mega_UCL['cross_attempted'].fillna(0)


In [125]:
new_mega_UCL.corr(numeric_only=True)['cross_attempted'].sort_values(ascending=False)


cross_attempted        1.000000
total_attempts         0.414355
match_played           0.411263
pass_attempted         0.410919
minutes_played         0.404717
fouls_suffered         0.393452
off_target             0.356608
t_lost                 0.221283
clearance_attempted    0.075990
position_binary        0.043055
Name: cross_attempted, dtype: float64

In [126]:
# Predictors
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)


,coef,std err,z,P>|z|
minutes_played,-0.0241,0.004,-5.451,0.000
match_played,0.0914,0.078,1.169,0.242
distance_covered,0.2406,0.042,5.793,0.000
fouls_suffered,0.1168,0.054,2.147,0.032
total_attempts,0.5410,0.100,5.432,0.000
off_target,-0.4032,0.157,-2.575,0.010
t_lost,0.0988,0.063,1.557,0.119
clearance_attempted,-0.3000,0.040,-7.409,0.000
pass_attempted,-0.0064,0.002,-2.783,0.005
cross_attempted,-0.1122,0.023,-4.823,0.000


In [127]:
cols_to_delete = ['match_played', 't_lost']

new_mega_UCL = new_mega_UCL.drop(columns=cols_to_delete, errors='ignore')


In [129]:
# 1. Remove any old versions if they exist
cols_to_drop = [c for c in new_mega_UCL.columns if c.startswith('cross_complted')]
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_drop, errors='ignore')

# 2. Extract from distribution dataset
distribution_crosscompleted = distribution_UCL[['player_name', 'club', 'cross_complted']]

# 3. Merge
new_mega_UCL = new_mega_UCL.merge(
    distribution_crosscompleted,
    on=['player_name', 'club'],
    how='left'
)

# 4. Fill missing with 0
new_mega_UCL['cross_complted'] = new_mega_UCL['cross_complted'].fillna(0)


In [130]:
new_mega_UCL.corr(numeric_only=True)['cross_complted'].sort_values(ascending=False)


cross_complted         1.000000
cross_attempted        0.912295
pass_attempted         0.376589
fouls_suffered         0.355868
minutes_played         0.353460
total_attempts         0.345169
off_target             0.306607
position_binary        0.060284
clearance_attempted    0.045371
Name: cross_complted, dtype: float64

In [131]:
# Predictors
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)


,coef,std err,z,P>|z|
minutes_played,-0.0258,0.005,-5.722,0.000
distance_covered,0.2787,0.038,7.302,0.000
fouls_suffered,0.1256,0.054,2.336,0.020
total_attempts,0.5772,0.104,5.574,0.000
off_target,-0.4318,0.159,-2.714,0.007
clearance_attempted,-0.2979,0.040,-7.401,0.000
pass_attempted,-0.0078,0.002,-3.520,0.000
cross_attempted,-0.1960,0.043,-4.595,0.000
cross_complted,0.2414,0.115,2.108,0.035


In [132]:
# 1. Remove any old versions if they exist
cols_to_drop = [c for c in new_mega_UCL.columns if c.startswith('freekicks_taken')]
new_mega_UCL = new_mega_UCL.drop(columns=cols_to_drop, errors='ignore')

# 2. Extract from distribution dataset
distribution_freekicks = distribution_UCL[['player_name', 'club', 'freekicks_taken']]

# 3. Merge
new_mega_UCL = new_mega_UCL.merge(
    distribution_freekicks,
    on=['player_name', 'club'],
    how='left'
)

# 4. Fill missing with 0
new_mega_UCL['freekicks_taken'] = new_mega_UCL['freekicks_taken'].fillna(0)


In [133]:
new_mega_UCL.corr(numeric_only=True)['freekicks_taken'].sort_values(ascending=False)


freekicks_taken        1.000000
pass_attempted         0.659888
minutes_played         0.608212
clearance_attempted    0.323378
cross_complted         0.313760
cross_attempted        0.277962
fouls_suffered         0.222932
off_target             0.046170
total_attempts         0.029051
position_binary       -0.252077
Name: freekicks_taken, dtype: float64

In [134]:
# Predictors
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)


,coef,std err,z,P>|z|
minutes_played,-0.0256,0.005,-5.627,0.000
distance_covered,0.2778,0.039,7.215,0.000
fouls_suffered,0.1348,0.055,2.435,0.015
total_attempts,0.5695,0.103,5.519,0.000
off_target,-0.4305,0.158,-2.726,0.006
clearance_attempted,-0.2968,0.040,-7.330,0.000
pass_attempted,-0.0071,0.002,-3.068,0.002
cross_attempted,-0.2006,0.043,-4.664,0.000
cross_complted,0.2676,0.120,2.234,0.025
freekicks_taken,-0.0327,0.043,-0.761,0.446


In [135]:
new_mega_UCL = new_mega_UCL.drop(columns=['freekicks_taken'], errors='ignore')

In [138]:
# full correlation matrix for all numeric features
corr_matrix = new_mega_UCL.corr(numeric_only=True)

corr_matrix


,minutes_played,position_binary,fouls_suffered,total_attempts,off_target,clearance_attempted,pass_attempted,cross_attempted,cross_complted
minutes_played,1.000000,-0.124257,0.551247,0.498447,0.471545,0.496135,0.860922,0.404717,0.353460
position_binary,-0.124257,1.000000,0.251948,0.317618,0.241476,-0.408729,-0.167090,0.043055,0.060284
fouls_suffered,0.551247,0.251948,1.000000,0.540556,0.510947,0.137384,0.439364,0.393452,0.355868
total_attempts,0.498447,0.317618,0.540556,1.000000,0.866831,0.007276,0.355774,0.414355,0.345169
off_target,0.471545,0.241476,0.510947,0.866831,1.000000,0.089541,0.360392,0.356608,0.306607
clearance_attempted,0.496135,-0.408729,0.137384,0.007276,0.089541,1.000000,0.498611,0.075990,0.045371
pass_attempted,0.860922,-0.167090,0.439364,0.355774,0.360392,0.498611,1.000000,0.410919,0.376589
cross_attempted,0.404717,0.043055,0.393452,0.414355,0.356608,0.075990,0.410919,1.000000,0.912295
cross_complted,0.353460,0.060284,0.355868,0.345169,0.306607,0.045371,0.376589,0.912295,1.000000


In [139]:
# Predictors
X_UCL_key = new_mega_UCL.drop(columns=['player_name', 'club', 'position', 'position_binary'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)

# Target
Y_UCL_key = new_mega_UCL['position_binary'].astype(int)

glm = sm.GLM(Y_UCL_key, X_UCL_key, family=sm.families.Binomial())
results = glm.fit()
summarize(results)

,coef,std err,z,P>|z|
minutes_played,-0.0258,0.005,-5.722,0.000
distance_covered,0.2787,0.038,7.302,0.000
fouls_suffered,0.1256,0.054,2.336,0.020
total_attempts,0.5772,0.104,5.574,0.000
off_target,-0.4318,0.159,-2.714,0.007
clearance_attempted,-0.2979,0.040,-7.401,0.000
pass_attempted,-0.0078,0.002,-3.520,0.000
cross_attempted,-0.1960,0.043,-4.595,0.000
cross_complted,0.2414,0.115,2.108,0.035
